In [44]:
import os 
import glob
import shutil
import uuid
from tqdm import tqdm
from PIL import Image

In [3]:
classes = sorted(os.listdir("../datasets/train/"))
classes

['Acanthastrea',
 'Acanthophyllia',
 'Acropora',
 'Alveopora & Goniopora',
 'Blastomussa',
 'Bubble Tip Anemone',
 'Bubble coral',
 'Candy cane coral',
 'Carpet Anemone',
 'Chalice',
 'Cyphastrea',
 'Discosoma Mushroom',
 'Elegance coral',
 'Euphyllia',
 'Favia',
 'Gorgonia',
 'Leptastrea',
 'Leptoseries',
 'Lobophyllia Trachyphyllia & Wellsophyllia',
 'Maze Brain coral Platygyra',
 'Mini Carpet Anemone',
 'Montipora',
 'Other',
 'Pavona',
 'Plate coral Fungia',
 'Porites',
 'Psammacora',
 'Rhodactis Mushroom',
 'Ricordea Mushroom',
 'Rock Flower Anemone',
 'Scolymia',
 'Scroll Corals Turbinaria',
 'Star polyps',
 'Styllopora & Pocillipora',
 'Sun coral',
 'Toadstool coral',
 'Tridacna clams',
 'Zoa']

In [4]:
for c in classes:
    try:
        os.mkdir(f"../datasets/data-lake/{c}")
    except:
        pass

In [67]:
from dask.distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://127.0.0.1:39591 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 33.69 GB


In [73]:
p = "../datasets/labeled"
target_extension = ".jpg"
extensions = ["*.jpg", "*.png"]

import pandas as pd
import dask.bag as db

def dasked_rename(img_path):
    try:
        img = Image.open(img_path)
        dirname = os.path.dirname(img_path)
        new_name = os.path.join(dirname, str(uuid.uuid4()).replace("-", "") + target_extension)
        img = img.convert("RGB")
        img.save(new_name)
        os.remove(img_path)
        return (img_path, 1, None)
    except Exception as ex:
        return (img_path, 0, str(ex))


files = []
for extension in extensions:
    pattern = os.path.join(p, "*", extension)
    files.extend(sorted(glob.glob(pattern, recursive=False)))
    

results = db.from_sequence(files).map(dasked_rename).compute()

df = pd.DataFrame(results, columns=["file", "success", "exception"])

In [69]:
client.close()

In [70]:
df.exception.value_counts()

Series([], Name: exception, dtype: int64)

In [72]:
p = "../datasets/labeled"

files = []
for extension in extensions:
    pattern = os.path.join(p, "*", extension)
    files.extend(sorted(glob.glob(pattern, recursive=False)))
    
    
print(files)

['../datasets/labeled/Acanthastrea/00036b3579854687b7ea2106fc56ce3a.jpg', '../datasets/labeled/Acanthastrea/003558f21c674fe7bcf1b72f8d994256.jpg', '../datasets/labeled/Acanthastrea/004ce4dc2fa5485292946ff0db439b9c.jpg', '../datasets/labeled/Acanthastrea/006a6a82bf594cf2a52f98b7bff6b0c9.jpg', '../datasets/labeled/Acanthastrea/006b579eae6444c5ba3a1abda53891c9.jpg', '../datasets/labeled/Acanthastrea/00739aec51594328a3de2da30604d8c3.jpg', '../datasets/labeled/Acanthastrea/0075c69a11cd4fffb121f719fb5b7894.jpg', '../datasets/labeled/Acanthastrea/008892e1e9de4c41aea091b373d62011.jpg', '../datasets/labeled/Acanthastrea/00f3fda8b4ae4accac842f9b3b96f619.jpg', '../datasets/labeled/Acanthastrea/00f64f06d9aa4198a088b98ac4ee049c.jpg', '../datasets/labeled/Acanthastrea/01034846b9ed4f049f6dca0f933f9cd5.jpg', '../datasets/labeled/Acanthastrea/01190f625c7440f7977049b86ac7555e.jpg', '../datasets/labeled/Acanthastrea/013d4af96b6840958bf13ffa09a986a4.jpg', '../datasets/labeled/Acanthastrea/01503b4da90f480b